# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [22]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv', null_values='NA', truncate_ragged_lines=True)
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv(
    'https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv',
    truncate_ragged_lines=True,
    null_values='NA', # Add null_values for weather data
    schema={
        'origin': pl.String,
        'year': pl.Int64,
        'month': pl.Int64,
        'day': pl.Int64,
        'hour': pl.Int64,
        'temp': pl.Float64,
        'dewp': pl.Float64,
        'humid': pl.Float64,
        'wind_dir': pl.Float64, # Changed to Float64 based on potential NA or decimal values
        'wind_speed': pl.Float64,
        'wind_gust': pl.Float64,
        'precip': pl.Float64,
        'pressure': pl.Float64,
        'visib': pl.Float64,
        'time_hour': pl.String # Explicitly load as String
    }
)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-471904640.py:36: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [23]:
result = ctx.execute("""
    SELECT DISTINCT carrier
    FROM airlines
""")

print("1.1 Unique carriers:")
print(result)

1.1 Unique carriers:
shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [25]:
# Write your SQL query here
result = ctx.execute("""
    SELECT dest, COUNT(*) AS flight_count
    FROM flights
    GROUP BY dest
    ORDER BY flight_count DESC
    LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬──────────────┐
│ dest ┆ flight_count │
│ ---  ┆ ---          │
│ str  ┆ u32          │
╞══════╪══════════════╡
│ ORD  ┆ 17283        │
│ ATL  ┆ 17215        │
│ LAX  ┆ 16174        │
│ BOS  ┆ 15508        │
│ MCO  ┆ 14082        │
│ CLT  ┆ 14064        │
│ SFO  ┆ 13331        │
│ FLL  ┆ 12055        │
│ MIA  ┆ 11728        │
│ DCA  ┆ 9705         │
└──────┴──────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [26]:
# Write your SQL query here
result = ctx.execute("""
    SELECT *
    FROM flights
    WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [27]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [28]:
# First, let's check what columns are available
# result = ctx.execute("""
#     SELECT *
#     FROM flights
#     LIMIT 5
# """)
# print(result)

# Now write your query to find busiest month
result = ctx.execute("""
    SELECT month, COUNT(*) AS flight_count
    FROM flights
    GROUP BY month
    ORDER BY flight_count DESC
    LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬──────────────┐
│ month ┆ flight_count │
│ ---   ┆ ---          │
│ i64   ┆ u32          │
╞═══════╪══════════════╡
│ 7     ┆ 29425        │
└───────┴──────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [29]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        carrier,
        SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS on_time_rate
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY carrier
    ORDER BY on_time_rate DESC
""")

print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ HA      ┆ 92.982456    │
│ US      ┆ 87.822674    │
│ AS      ┆ 86.797753    │
│ AA      ┆ 84.071293    │
│ DL      ┆ 83.681246    │
│ …       ┆ …            │
│ FL      ┆ 73.32915     │
│ WN      ┆ 73.102706    │
│ F9      ┆ 71.847507    │
│ YV      ┆ 71.376147    │
│ EV      ┆ 69.538126    │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [30]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    a.name,
    f.flight,
    f.origin,
    f.dest
FROM flights AS f
JOIN airlines AS a
  ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ name                   ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [31]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        f.carrier,
        AVG(2013 - p.year) AS average_plane_age
    FROM flights AS f
    JOIN planes AS p
        ON f.tailnum = p.tailnum
    WHERE p.year IS NOT NULL -- Exclude planes with unknown manufacture year
    GROUP BY f.carrier
    ORDER BY average_plane_age DESC
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────────┐
│ carrier ┆ average_plane_age │
│ ---     ┆ ---               │
│ str     ┆ f64               │
╞═════════╪═══════════════════╡
│ MQ      ┆ 35.319            │
│ AA      ┆ 25.869426         │
│ DL      ┆ 16.372169         │
│ UA      ┆ 13.207691         │
│ FL      ┆ 11.385829         │
│ …       ┆ …                 │
│ B6      ┆ 6.686702          │
│ F9      ┆ 4.87874           │
│ VX      ┆ 4.473643          │
│ AS      ┆ 3.33662           │
│ HA      ┆ 1.548387          │
└─────────┴───────────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [33]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
# print(result)

# Now write your join query
result = ctx.execute("""
SELECT
    f.*,
    w.year AS weather_year, -- Alias year from weather
    w.month AS weather_month, -- Alias month from weather
    w.day AS weather_day, -- Alias day from weather
    w.hour AS weather_hour, -- Alias hour from weather
    w.temp,
    w.dewp,
    w.humid,
    w.wind_dir,
    w.wind_speed,
    w.wind_gust,
    w.precip,
    w.pressure,
    w.visib,
    w.time_hour AS weather_time_hour -- Alias time_hour from weather
FROM flights AS f
JOIN weather AS w
  ON f.origin = w.origin AND f.time_hour = w.time_hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result)

shape: (4_938, 33)
┌──────┬───────┬─────┬──────────┬───┬────────┬──────────┬───────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ precip ┆ pressure ┆ visib ┆ weather_time_hour       │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---    ┆ ---      ┆ ---   ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ f64    ┆ f64      ┆ f64   ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪════════╪══════════╪═══════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 2217     ┆ … ┆ 0.0    ┆ 1015.3   ┆ 10.0  ┆ 2013-01-02 02:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 2326     ┆ … ┆ 0.0    ┆ 1015.3   ┆ 10.0  ┆ 2013-01-02 02:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1238     ┆ … ┆ 0.0    ┆ 1016.9   ┆ 10.0  ┆ 2013-01-04 16:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1400     ┆ … ┆ 0.0    ┆ 1015.1   ┆ 10.0  ┆ 2013-01-04 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1455     ┆ … ┆ 0.0    ┆ 1014.9   ┆ 10.0  ┆ 2013-01-04 19:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …      ┆ …        ┆ …  

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [34]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        p.manufacturer,
        p.model,
        COUNT(f.flight) AS flight_count
    FROM flights AS f
    JOIN planes AS p
        ON f.tailnum = p.tailnum
    GROUP BY p.manufacturer, p.model
    ORDER BY flight_count DESC
    LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬──────────────┐
│ manufacturer                  ┆ model           ┆ flight_count │
│ ---                           ┆ ---             ┆ ---          │
│ str                           ┆ str             ┆ u32          │
╞═══════════════════════════════╪═════════════════╪══════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278        │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027        │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716        │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553        │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051        │
│ BOEING                        ┆ 737-824         ┆ 13809        │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807        │
│ BOEING                        ┆ 737-7H4         ┆ 10389        │
│ BOEING                        ┆ 757-222         ┆ 9150         │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88        

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [37]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        origin_grouped.name AS origin_name,
        dest_grouped.name AS dest_name,
        route_performance.total_flights,
        route_performance.average_departure_delay,
        route_performance.percentage_delayed_30_plus
    FROM (
        SELECT
            f.origin,
            f.dest,
            COUNT(*) AS total_flights,
            AVG(f.dep_delay) AS average_departure_delay,
            SUM(CASE WHEN f.dep_delay > 30 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS percentage_delayed_30_plus
        FROM flights AS f
        WHERE f.dep_delay IS NOT NULL
        GROUP BY f.origin, f.dest
    ) AS route_performance
    JOIN airports AS origin_grouped
      ON route_performance.origin = origin_grouped.faa
    JOIN airports AS dest_grouped
      ON route_performance.dest = dest_grouped.faa
    ORDER BY route_performance.total_flights DESC
    LIMIT 10
""")

print(result)

shape: (10, 5)
┌────────────────┬─────────────────────┬───────────────┬─────────────────────┬─────────────────────┐
│ origin_name    ┆ dest_name           ┆ total_flights ┆ average_departure_d ┆ percentage_delayed_ │
│ ---            ┆ ---                 ┆ ---           ┆ elay                ┆ 30_plus             │
│ str            ┆ str                 ┆ u32           ┆ ---                 ┆ ---                 │
│                ┆                     ┆               ┆ f64                 ┆ f64                 │
╞════════════════╪═════════════════════╪═══════════════╪═════════════════════╪═════════════════════╡
│ John F Kennedy ┆ Los Angeles Intl    ┆ 11196         ┆ 8.522508            ┆ 9.88746             │
│ Intl           ┆                     ┆               ┆                     ┆                     │
│ La Guardia     ┆ Hartsfield Jackson  ┆ 10082         ┆ 11.448621           ┆ 12.467764           │
│                ┆ Atlanta Int…        ┆               ┆                    

In [ ]:
# Write your SQL query here
result = ctx.execute("""
-- Your query here
""")

# print(result)

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [ ]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here

In [38]:
# Choose one of the previous SQL queries and implement it here using Polars.
# For example, you could implement Exercise 1.2 (Top 10 destinations) or 3.1 (Flights with airline names).

# Your Polars code here